In [23]:
import numpy as np
import pandas as pd

import re
from nltk.corpus import stopwords
import nltk
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel, LsiModel, LdaModel, HdpModel
import pyLDAvis.gensim
from nltk.stem import WordNetLemmatizer
stop_words = stopwords.words('english')

In [2]:
df = pd.read_excel('RealEstate.xlsx')
df

,web_id,property_text_en
0,7023850,Smart Business Real Estate is proud to present...
1,6905802,My Island Real Estate is proud to present 1 Be...
2,6998469,"New on the market, amazing opportunity for inv..."
3,6766136,1 Bedroom Apartment in Marina Diamond 2\n\n- 1...
4,6940244,No penalty for early contract termination!<br ...
...,...,...
756,6203680,Castles Plaza Real Estate is pleased to bring ...
757,7090736,The Noble House Real Estate is proud to presen...
758,7081426,Better Homes would like to present this type 2...
759,7123539,Morgan‚Äôs International Realty is proud to pr...


#### Inspecting the first 8 entries

In [3]:
# Inspecting the first 8 entries
for entry in df.loc[0:8,'property_text_en']:
    print(entry)
    print()
    print()

Smart Business Real Estate is proud to present2Bedrooms for sale locatedat Dubai MarinaElite Residenceon mid floor over looking court yard.\n\nProperty currently Rentedat AED 110,000 till25 of June 2020.\n\nElite Residenceis ideally locatedat the middle of Dubai Marina area , close to the Tran stationas well Metro stationnear by .surrounding by lots of cafe and restaurants , Marina Walkpromenade , across streetfew5 *Hotels. Dubai Marinais very popularand most desirable place to live.\n\nAt the building available facilities:\nGym\nPool\nSecurity 24/7\nCovered parking .\n\nFor more information or viewing appointment, call:\nSmart Business Real Estate LLC\nLandline:+971 4 5808525 / 0503474100\nEmail: info@sbrealestate.ae\nBRN 11575\nORN 2104


My Island Real Estate is proud to present 1 Bedroom Apt. in Dubai Marina with Partial Sea View\n\n\nDubai Marina\nPrincess Tower\n1 Bedroom Apartment\nPartial Sea View\n922.68 sqft\nUnfurnished Apartment\n\n\n\nFor more details, please call 056-9576

#### Here we notice the text includes a lot of:
- html
- '\n' which means new line
- phone numbers starting with + sign 
    - many formats
    - +xxx xx xxx xxxx
    - +xxx x xxx xxxx
    - +xxxxxxxxxxxx
    - xxxxx (continuous block of numbers)
    - xxxx,xxxx
- special characters
- emails

So we need to preprocess the data and clean it

Method for preprocessing each document

In [4]:

def preprocessing(document):
    document = document.lower()

    #removing the html
    document = re.sub(r'(\<[ ]?[a-z]+>|\<\/[a-z]+\>)', ' ', document)
    # noticed that '<' stayed for some reason, so removing them
    document = re.sub(r'<', ' ', document)
    #
    # removing emails
    document = re.sub(r'\S+@\S+', ' ', document)
    # removing \n
    document = re.sub(r'(\\n)', " ", document)
    # removing numbers of this formats:
    # - xxxx,xxxx (numbers with commas)
    document = re.sub(r'(\d+),(\d+),?(\d*)', " ", document)
    # removing numbers of all other formats
    document = re.sub(r'(\+)?\d+[ ]?\d*[ ]?\d*[ ]?\d*', ' ', document)
    # removing characters like ? ! : ; \n \
    document = re.sub('[*.?!,:;/&]', ' ', document)
    # removal of --
    document = re.sub(r'--'," ", document)
    # removal of word 'dubai'
    #document = re.sub(r'dubai'," ", document)

    document = " ".join([word for word in document.split() if len(word) > 3 ])
    
    document = " ".join([word for word in document.split() if word not in stop_words])

    document = " ".join([WordNetLemmatizer().lemmatize(word) for word in document.split()])

    return document

In [5]:
#df['property_text_en'][0]

In [6]:
# apply the method on the documents column
df['property_text_en'] = df['property_text_en'].apply(preprocessing)
for document in df['property_text_en']:
    pass

Lets see how the text changed after preprocessing

In [7]:
for document in df.loc[0:8,'property_text_en']:
    print(document)
    print()
    print()

smart business real estate proud present bedroom sale locatedat dubai marinaelite residenceon floor looking court yard property currently rentedat till june elite residenceis ideally locatedat middle dubai marina area close tran stationas well metro stationnear surrounding lot cafe restaurant marina walkpromenade across streetfew hotel dubai marinais popularand desirable place live building available facility pool security covered parking information viewing appointment call smart business real estate landline email


island real estate proud present bedroom dubai marina partial view dubai marina princess tower bedroom apartment partial view sqft unfurnished apartment detail please call email marina dubai marina awe-inspiring city-within-a city delight resident cosmopolitan free-spirited atmosphere unique invigorating lifestyle urban center water comparable exclusive waterfront development world leading city unlike anywhere else dubai middle east even world island real estate island re

In [8]:
#re.sub(r'[\.\?\!\,\:\;\\\\/"]', '', df['property_text_en'][0])

In [9]:
df.loc[:,'property_text_en']

0      smart business real estate proud present bedro...
1      island real estate proud present bedroom dubai...
2      market amazing opportunity investor apartment ...
3      bedroom apartment marina diamond bedroom apart...
4      penalty early contract termination price varie...
                             ...                        
756    castle plaza real estate pleased bring market ...
757    noble house real estate proud present full flo...
758    better home would like present type three bedr...
759    morgan‚äôs international realty proud present ...
760    bedroom reem arabian ranch directly opposite p...
Name: property_text_en, Length: 761, dtype: object

In [10]:
text = df['property_text_en'].tolist()
text = [nltk.word_tokenize(doc) for doc in text]

In [11]:
# preparing inputs for the LSI model
dictionary = corpora.Dictionary(text)

In [12]:
corpus = [dictionary.doc2bow(doc) for doc in text]

In [13]:
lsimodel = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [14]:
lsimodel.show_topics(num_topics=10)

[(0,
  '0.405*"dubai" + 0.393*"marina" + 0.221*"apartment" + 0.212*"tower" + 0.195*"bedroom" + 0.182*"room" + 0.178*"view" + 0.172*"area" + 0.165*"property" + 0.157*"floor"'),
 (1,
  '-0.363*"marina" + -0.300*"dubai" + 0.268*"bedroom" + 0.258*"property" + 0.213*"room" + -0.167*"tower" + 0.161*"study" + 0.141*"bathroom" + 0.141*"garden" + 0.138*"villa"'),
 (2,
  '-0.504*"tower" + -0.341*"princess" + -0.301*"floor" + 0.289*"marina" + 0.175*"dubai" + -0.159*"residential" + 0.152*"property" + -0.152*"tallest" + -0.130*"world" + 0.125*"walk"'),
 (3,
  '0.432*"elite" + 0.425*"residence" + -0.232*"estate" + -0.232*"real" + 0.212*"room" + -0.195*"property" + -0.172*"dubai" + -0.160*"marina" + -0.150*"tower" + 0.145*"pool"'),
 (4,
  '-0.404*"real" + -0.402*"estate" + -0.314*"elite" + -0.298*"residence" + 0.174*"marina" + -0.165*"service" + 0.162*"room" + -0.154*"property" + 0.149*"apartment" + 0.127*"bedroom"'),
 (5,
  '-0.380*"room" + -0.346*"dubai" + 0.198*"marina" + 0.166*"walk" + 0.160*"tow

Lets try it with bigrams instead 

In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(text, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

bigramed_words = make_bigrams(text)

bigram_dictionary = corpora.Dictionary(text)
bigram_corpus = [dictionary.doc2bow(doc) for doc in text]

lsimodel = LsiModel(corpus=bigram_corpus, num_topics=10, id2word=bigram_dictionary)
lsimodel.print_topics(num_topics=10)

[(0,
  '0.405*"dubai" + 0.393*"marina" + 0.221*"apartment" + 0.212*"tower" + 0.195*"bedroom" + 0.182*"room" + 0.178*"view" + 0.172*"area" + 0.165*"property" + 0.157*"floor"'),
 (1,
  '-0.363*"marina" + -0.300*"dubai" + 0.268*"bedroom" + 0.258*"property" + 0.213*"room" + -0.167*"tower" + 0.161*"study" + 0.141*"bathroom" + 0.141*"garden" + 0.138*"villa"'),
 (2,
  '-0.504*"tower" + -0.341*"princess" + -0.301*"floor" + 0.289*"marina" + 0.175*"dubai" + -0.159*"residential" + 0.152*"property" + -0.152*"tallest" + -0.130*"world" + 0.125*"walk"'),
 (3,
  '0.432*"elite" + 0.425*"residence" + -0.232*"estate" + -0.232*"real" + 0.212*"room" + -0.195*"property" + -0.172*"dubai" + -0.160*"marina" + -0.150*"tower" + 0.145*"pool"'),
 (4,
  '0.404*"real" + 0.402*"estate" + 0.314*"elite" + 0.298*"residence" + -0.174*"marina" + 0.165*"service" + -0.162*"room" + 0.154*"property" + -0.149*"apartment" + -0.127*"bedroom"'),
 (5,
  '0.380*"room" + 0.346*"dubai" + -0.198*"marina" + -0.166*"walk" + -0.160*"towe

In [15]:
coherence_model_lsi = CoherenceModel(model=lsimodel, texts=text, dictionary=dictionary, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score: ', coherence_lsi)



Coherence Score:  0.4441955226461918


So we got a coherence score of 0.4


### Using LDA model

In [16]:
lda_model = LdaModel(corpus = corpus, id2word= dictionary, num_topics= 10)
lda_model.show_topics()

[(0,
  '0.032*"marina" + 0.022*"dubai" + 0.015*"room" + 0.015*"apartment" + 0.013*"pool" + 0.013*"residence" + 0.013*"floor" + 0.012*"bedroom" + 0.012*"elite" + 0.012*"tower"'),
 (1,
  '0.025*"dubai" + 0.015*"marina" + 0.014*"property" + 0.014*"bedroom" + 0.014*"area" + 0.012*"apartment" + 0.010*"room" + 0.010*"offer" + 0.010*"view" + 0.008*"elite"'),
 (2,
  '0.020*"property" + 0.017*"marina" + 0.016*"estate" + 0.015*"dubai" + 0.015*"apartment" + 0.014*"real" + 0.013*"room" + 0.012*"view" + 0.012*"bedroom" + 0.009*"elite"'),
 (3,
  '0.038*"marina" + 0.025*"dubai" + 0.019*"apartment" + 0.014*"tower" + 0.012*"view" + 0.011*"bedroom" + 0.010*"room" + 0.010*"area" + 0.009*"floor" + 0.008*"walk"'),
 (4,
  '0.029*"dubai" + 0.025*"marina" + 0.017*"area" + 0.017*"tower" + 0.015*"room" + 0.013*"princess" + 0.013*"bedroom" + 0.012*"apartment" + 0.011*"view" + 0.010*"property"'),
 (5,
  '0.024*"dubai" + 0.017*"marina" + 0.016*"floor" + 0.015*"room" + 0.014*"bedroom" + 0.014*"property" + 0.012*"ar

In [17]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.023548  0.012779       1        1  16.190327
4      0.022682  0.007763       2        1  16.062807
0      0.044270 -0.010025       3        1  14.764574
5      0.010547  0.013929       4        1  13.141121
8      0.017611 -0.010882       5        1  10.576405
3      0.012789  0.002474       6        1   9.012095
1     -0.021166 -0.052271       7        1   6.970097
2     -0.028036  0.012984       8        1   6.655417
7     -0.045230 -0.011952       9        1   4.596169
9     -0.037015  0.035202      10        1   2.030986, topic_info=          Term         Freq        Total Category  logprob  loglift
39    property   900.000000   900.000000  Default  30.0000  30.0000
17      estate   587.000000   587.000000  Default  29.0000  29.0000
41        real   598.000000   598.000000  Default  28.0000  28.0000
106      tower   975.000000   975.000000  Default  27.0000  27.0000
64   apartment  1022.000000  1022.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
111       view     8.094111   913.512734  Topic10  -5.2562  -0.8295
139    located     6.372788   421.189446  Topic10  -5.4953  -0.2944
386       palm     5.970821   339.224368  Topic10  -5.5605  -0.1431
15       elite     6.755059   660.777830  Topic10  -5.4371  -0.6865
36        pool     6.292105   707.996918  Topic10  -5.5081  -0.8265

[828 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
769       7  0.471900  -bathroom
2480      6  0.575283    -guests
3500      5  0.402333    -public
3500      9  0.201167    -public
770       7  0.470601    -status
...     ...       ...        ...
2469      5  0.075183     ‚ô¶‚ô¶
2469      6  0.075183     ‚ô¶‚ô¶
2469      7  0.075183     ‚ô¶‚ô¶
2469      8  0.075183     ‚ô¶‚ô¶
2469      9  0.375913     ‚ô¶‚ô¶

[2614 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 1, 6, 9, 4, 2, 3, 8, 10])

In [18]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.45219252639937857


In [19]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)
hdpmodel.show_topics()

/Users/zeinathabet/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


[(0,
  '0.015*dubai + 0.015*marina + 0.011*tower + 0.009*floor + 0.009*apartment + 0.008*bedroom + 0.008*area + 0.008*view + 0.008*room + 0.007*princess + 0.007*property + 0.006*real + 0.006*estate + 0.006*elite + 0.006*residence + 0.006*pool + 0.005*residential + 0.005*call + 0.004*parking + 0.004*located'),
 (1,
  '0.013*dubai + 0.011*marina + 0.005*view + 0.005*apartment + 0.005*tower + 0.005*bedroom + 0.005*property + 0.003*located + 0.003*kitchen + 0.003*area + 0.003*building + 0.003*floor + 0.003*residential + 0.003*estate + 0.003*real + 0.003*residence + 0.003*elite + 0.002*princess + 0.002*parking + 0.002*walk'),
 (2,
  '0.010*marina + 0.008*dubai + 0.005*apartment + 0.004*property + 0.004*view + 0.004*tower + 0.004*bedroom + 0.003*real + 0.003*pool + 0.003*residence + 0.003*floor + 0.003*elite + 0.003*offer + 0.003*area + 0.003*room + 0.003*estate + 0.003*parking + 0.003*call + 0.003*beach + 0.002*viewing'),
 (3,
  '0.008*dubai + 0.006*bedroom + 0.006*marina + 0.005*apartment 

In [20]:
hdpmodel_coherence = CoherenceModel(model = hdpmodel, texts= text, dictionary= dictionary, coherence='c_v')
hdp_coherence = hdpmodel_coherence.get_coherence()
print('Coherence score: ',hdp_coherence)

Coherence score:  0.6946988792668161
